# 🔧 TimesFM RUL Prediction on Simulated Vibration Data
This notebook demonstrates how to use TimesFM for Remaining Useful Life (RUL) prediction using a simulated dataset.

In [ ]:
# 📦 Install necessary libraries (run once)
!pip install timesfm[torch] torch pandas matplotlib

In [2]:
import numpy as np
import pandas as pd

# Function to simulate vibration data with RUL labels
def simulate_vibration_data(samples=500, sequence_length=128, fail_at_end=True):
    data = []
    rul = []

    for i in range(samples):
        # Simulate increasing degradation if fail_at_end is True
        degradation = i / samples if fail_at_end else 0.5
        noise = np.random.normal(loc=0.0, scale=0.1, size=sequence_length)
        trend = np.linspace(0, degradation, sequence_length)
        vibration = noise + trend
        data.append(vibration)
        rul.append(samples - i if fail_at_end else np.random.randint(50, 200))

    return np.array(data), np.array(rul)

# Generate the data
sim_data, sim_rul = simulate_vibration_data()

# Convert to DataFrame and add RUL column
df_simulated = pd.DataFrame(sim_data)
df_simulated['RUL'] = sim_rul

# Save to CSV
csv_path = "simulated_rul_data.csv"
df_simulated.to_csv(csv_path, index=False)

csv_path


'simulated_rul_data.csv'

In [3]:


# Load the dataset
df = pd.read_csv('simulated_rul_data.csv')  # <-- Replace with actual file path if needed

# Separate features and RUL
X = df.drop(columns=['RUL']).values
y = df['RUL'].values

In [5]:
# 📥 Prepare input for TimesFM
import torch

# Normalize and convert to tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = X_tensor.unsqueeze(1)  # shape: (batch, sequence_length) to (batch, 1, sequence_length)

In [11]:
import numpy as np
import pandas as pd
import timesfm
import matplotlib.pyplot as plt

context_length = 256
horizon_length = 64

print(f"Initializing TimesFM with context_len={context_length}, horizon_len={horizon_length}")

# Explicitly specify the backend to "torch"
tfm = timesfm.TimesFm(
    hparams=timesfm.TimesFmHparams(
        # backend="cpu", # This line might implicitly trigger JAX if the PyTorch backend isn't fully isolated
        # Instead, specify the PyTorch backend directly if using the PyTorch checkpoint
        backend="torch", # Explicitly use the PyTorch backend
        context_len=context_length,
        horizon_len=horizon_length,
        input_patch_len=32,
        output_patch_len=128,
        num_layers=20,
        model_dims=1280,
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id="google/timesfm-1.0-200m-pytorch"
    )
)

print("TimesFM model loaded successfully!")

# ... (rest of your code)

Initializing TimesFM with context_len=256, horizon_len=64


LocalEntryNotFoundError: An error happened while trying to locate the files on the Hub and we cannot find the appropriate snapshot folder for the specified revision on the local disk. Please check your internet connection and try again.

In [22]:
import timesfm

# Initialize TimesFM with PyTorch backend
tfm = timesfm.TimesFm(
    input_patch_len=32,
output_patch_len=128,
num_layers=20,
model_dims=1280,
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

TypeError: TimesFmBase.__init__() got an unexpected keyword argument 'input_patch_len'

In [21]:
import timesfm
import pandas as pd
import numpy as np

# Initialize TimesFM with PyTorch backend
tfm = timesfm.TimesFm(
    context_len=512,  # Maximum context length for input time-series
    backend="cpu"  # Use "mps" for GPU acceleration on Apple silicon if available
)

# Load the pretrained checkpoint
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

# Load your dataset (e.g., simulated vibration data or NASA bearing dataset)
df = pd.read_csv("path/to/vibration_data.csv")  # Adjust path to your dataset
time_series = df["amplitude"].values  # Univariate series (e.g., vibration amplitude)
frequency = 0  # High-frequency data (e.g., 1-second windows)

# Ensure contiguous data
assert not np.any(np.isnan(time_series)), "Data must be contiguous (no NaNs)"

# Perform forecasting
forecast = tfm.forecast(
    inputs=[time_series],  # List of time-series (single series for univariate)
    freq=[frequency],  # Frequency indicator
    horizon_len=128  # Forecast horizon (e.g., 128 time points)
)

# Print forecast
print("Forecasted RUL values:", forecast[0])

# Optional: Evaluate and visualize
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Assuming ground truth for the forecast horizon
ground_truth = df["rul"].values[-128:]  # Last 128 points
mae = mean_absolute_error(ground_truth, forecast[0])
print(f"MAE: {mae:.4f}")

plt.plot(range(len(time_series)), time_series, label="Historical")
plt.plot(range(len(time_series), len(time_series) + 128), forecast[0], label="Forecast")
plt.xlabel("Time")
plt.ylabel("Vibration Amplitude")
plt.legend()
plt.show()

TypeError: TimesFmBase.__init__() got an unexpected keyword argument 'context_len'

In [15]:
# 🧠 Run inference (Placeholder)
# TimesFM is typically used for forecasting; here we illustrate a mock output for RUL estimation
predicted_rul = model.generate(X_tensor)  # This line is symbolic — replace with forecast logic

NameError: name 'model' is not defined